In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import random

In [ ]:
class DataLoader:
    def __init__(self,
                 datapath,
                 shuffle=True):
        self.shuffle = shuffle
        self.data = {}

        for root, _, files in os.walk(datapath):
            for name in files:
                filename = name.split('.')
                self.data[filename[0] + filename[1]] = np.load(os.path.join(root, name))

    def __call__(self):
        keys = list(self.data.keys())
        if self.shuffle:
            random.shuffle(keys)
        # итерация по ключам в словаре self.data
        for key in keys:
            # если маска целевого значения пустая, то пропускаем пример
            if np.all(self.data[key][:,:,-1,:,:] == 0.):
                continue
            seq = self.__getSequence(key)
            arrays = []
            badCount = 0
            # итерация по историческим данным
            for item in seq:
                # некоторые исторические данные могут отсутствовать
                try:
                    if np.all(self.data[item][:,:,-1,:,:] == 0.):
                        badCount += 1
                    arrays.append(self.data[item])
                except KeyError:
                    # print(f'No key: {item}')
                    badCount += 1
                    arrays.append(np.zeros_like(self.data[key]))
            # если пропусков в данных больше чем 30%, то пропускаем пример
            # print(f'Bad count: {badCount}')
            if badCount / len(arrays) > 0.3:
                continue
            else:
                x = np.concatenate(arrays, axis=1)
                y = self.data[key]
                for beam in range(16):
                    yield x[:,:,:,beam,0], y[:,:,:-1,beam,0]
    
    def __getSequence(self, key):
        keyDT = datetime.strptime(key, '%Y%m%d%H%M')
        # список массивов периодов за неделю до целевого массива
        weekBefore = []
        for i in range(24*7, 0, -2):
            hoursBefore = (keyDT-timedelta(hours=i)).strftime('%Y%m%d%H%M')
            weekBefore.append(hoursBefore)
        return weekBefore

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
batch_size = 1

train_loader = DataLoader('drive/MyDrive/2002-train')
# val_loader = DataLoader('drive/MyDrive/2002-val')

train_dataset = tf.data.Dataset.from_generator(train_loader,
                                         output_types=(tf.float64, tf.float64)).batch(batch_size)
# val_dataset = tf.data.Dataset.from_generator(val_loader,
#                                          output_types=(tf.float64, tf.float64)).batch(batch_size)

In [ ]:
# for x, y in train_loader:
#     print(x.shape, y.shape)
#     new_x = model(x.reshape([1] + list(x.shape)))

#     fig, axs = plt.subplots(2, 2)

#     axs[0, 0].imshow(x[:, :60, -1])
#     axs[0, 0].set_title("x at 0")

#     axs[1, 0].imshow(new_x[0, 0, :, :, -1])
#     axs[1, 0].set_title("new_x at 0")

#     axs[0, 1].imshow(x[:, 60*49:60*50, -1])
#     axs[0, 1].set_title("x at 50")

#     axs[1, 1].imshow(new_x[0, 49, :, :, -1])
#     axs[1, 1].set_title("new_x at 50")

#     plt.show()

#     break

In [ ]:
def make_generator_model():
    inp = tf.keras.layers.Input(shape=(70, 5040, 7))

    reshape = tf.keras.layers.Reshape(target_shape=(70, 5040//60, 60, 7))(inp)

    permute = tf.keras.layers.Permute((2,1,3,4))(reshape)

    x = tf.keras.layers.ConvLSTM2D(
        filters=64,
        kernel_size=(1, 1),
        # padding="same",
        return_sequences=True,
        data_format='channels_last',
        # activation="relu",
    )(permute)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.ConvLSTM2D(
        filters=64,
        kernel_size=(3, 3),
        # padding="same",
        return_sequences=True,
        data_format='channels_last',
        # activation="relu",
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.ConvLSTM2D(
        filters=64,
        kernel_size=(5, 5),
        # padding="same",
        return_sequences=True,
        data_format='channels_last',
        # activation="relu",
    )(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.Conv3D(
        filters=6, kernel_size=(7, 8, 8)
    )(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.Conv3D(
        filters=6, kernel_size=(28, 10, 10)
    )(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.Conv3D(
        filters=6, kernel_size=(51, 14, 14)
    )(x)
    x = tf.keras.layers.LeakyReLU(0.3)(x)

    x = tf.keras.layers.Conv3DTranspose(
        filters=6, kernel_size=(1, 36, 36), activation="linear"
    )(x)

    x = tf.keras.layers.Reshape(target_shape=(70, 60, 6))(x)

    model = tf.keras.models.Model(inp, x)

    return model

# model = make_generator_model()
# model.summary()

In [ ]:
from tensorflow import keras

def make_discriminator_model():
    hist_inp = keras.layers.Input(shape=(70, 5040, 7))
    hist_x = keras.layers.Conv2D(filters=6, kernel_size=(5, 60))(hist_inp)
    hist_x = keras.layers.BatchNormalization()(hist_x)
    hist_x = keras.layers.LeakyReLU(0.3)(hist_x)

    hist_x = keras.layers.Conv2D(filters=16, kernel_size=(5, 60*12), strides=(2,2))(hist_x)
    hist_x = keras.layers.BatchNormalization()(hist_x)
    hist_x = keras.layers.LeakyReLU(0.3)(hist_x)

    # model = keras.models.Model(hist_inp, hist_x, name='discriminator')
    # return model

    gen_out_inp = keras.layers.Input(shape=(70, 60, 6))
    gen_out_x = keras.layers.Conv2D(filters=6, kernel_size=(5, 5))(gen_out_inp)
    gen_out_x = keras.layers.BatchNormalization()(gen_out_x)
    gen_out_x = keras.layers.LeakyReLU(0.3)(gen_out_x)

    gen_out_x = keras.layers.Conv2D(filters=16, kernel_size=(5, 5), strides=(2,2))(gen_out_x)
    gen_out_x = keras.layers.BatchNormalization()(gen_out_x)
    gen_out_x = keras.layers.LeakyReLU(0.3)(gen_out_x)

    # model = keras.models.Model(gen_out_inp, gen_out_x)
    # return model

    joined = keras.layers.Concatenate(axis=2)([hist_x, gen_out_x])
    joined = keras.layers.Conv2D(filters=16, kernel_size=(1, 5))(joined)
    joined = keras.layers.BatchNormalization()(joined)
    joined = keras.layers.LeakyReLU(0.3)(joined)

    joined = keras.layers.Conv2D(filters=16, kernel_size=(1, 5), strides=(1,2))(joined)
    joined = keras.layers.BatchNormalization()(joined)
    joined = keras.layers.LeakyReLU(0.3)(joined)

    joined = keras.layers.Conv2D(filters=6, kernel_size=(1, 5), strides=(1,2))(joined)
    joined = keras.layers.BatchNormalization()(joined)
    joined = keras.layers.LeakyReLU(0.3)(joined)

    joined = keras.layers.Conv2D(filters=6, kernel_size=(1, 5),strides=(1,2))(joined)
    joined = keras.layers.BatchNormalization()(joined)
    joined = keras.layers.LeakyReLU(0.3)(joined)

    joined = keras.layers.Conv2D(filters=6, kernel_size=(1, 5),strides=(1,2))(joined)
    joined = keras.layers.BatchNormalization()(joined)
    joined = keras.layers.LeakyReLU(0.3)(joined)

    joined = keras.layers.Flatten()(joined)
    joined = keras.layers.Dense(60, activation='sigmoid')(joined)
    
    model = keras.models.Model([hist_inp, gen_out_inp], joined, name='discriminator')
    return model

# model = make_discriminator_model()
# model.summary()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
generator = make_generator_model()
discriminator = make_discriminator_model()

gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
gen_mae_tracker = keras.metrics.Mean(name="generator_mae")

In [ ]:
@tf.function
def train_step(x, y):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(x, training=True)

      real_output = discriminator([x, y], training=True)
      fake_output = discriminator([x, generated_images], training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    #   tf.print(f'gen_loss: {gen_loss}, disc_loss: {disc_loss}')

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
from tqdm import tqdm
def train(dataset, epochs):
    for epoch in range(epochs):
        for x, y in dataset:
            train_step(x, y)

In [ ]:
train(train_dataset, 100)

KeyboardInterrupt: ignored